# Childrens Act Disposals

In [18]:
# libraries
import pandas as pd
import pydbtools as pydb
import boto3
from datetime import datetime
#import re
import os  # for file paths
import awswrangler as wr

In [19]:
# Structured  dataframes
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

In [20]:
# Assigning databases and S3 bucket
db1 = "familyman_dev_v2"
fcsq_db = "fcsq"
# create path for within athena FCSQ database in the S3 folder
fcsq_db_path = f"s3://alpha-family-data/fcsq_processing/CA_disps/"
s3 = boto3.resource("s3")
bucket = s3.Bucket("alpha-family-data")
snapshot_date = "2021-11-11"

### Orders made

In [21]:
pydb.create_temp_table( 
f"""
  SELECT 
    DISTINCT
    case_number,
    disp_date,
    Year,
    Quarter,
    case_type,
    receipt_date,
    entry_date,
    event,
    event_model,
    field_model,
    order_type,
    disp_type_code,
    disp_type,
    order_code,
    order_desc,
    creating_court
  FROM 
    {fcsq_db}.ca_disps a 
  WHERE 
    disp_type_code = 1
""",

"ca_disp_order_type")

In [13]:
pydb.read_sql_query ("select count (*) count from __temp__.ca_order_type where year = 2020 and case_type = 'P' and order_code = 29 ")

,count
0,46368


### Disposal Events

In [22]:
pydb.create_temp_table( 
f"""
  SELECT 
    DISTINCT
    case_number,
    disp_date,
    Year,
    Quarter,
    case_type,
    receipt_date,
    entry_date,
    event,
    event_model,
    field_model,
    order_type,
    disp_type_code,
    disp_type,
    order_code,
    order_desc,
    creating_court,
    FO_value
  FROM 
    {fcsq_db}.ca_disps a 
  WHERE 
    order_code < 40
""",

"ca_disp_events")

In [21]:
pydb.read_sql_query ("select count (*) count from __temp__.ca_disp_events where year = 2011 and case_type = 'P'")

,count
0,122138


### Create flag for earliest/latest final order, non final order date - this can be moved to earier (or final order extraction can be moved to here)

In [71]:
pydb.create_temp_table(
f"""
Select
    *,
    ROW_NUMBER() OVER(PARTITION BY case_number, fo_value
                       ORDER BY case_number, disp_date ASC) 
      AS min_date,
    ROW_NUMBER() OVER(PARTITION BY case_number, fo_value
                       ORDER BY case_number, disp_date DESC) 
      AS max_date  
FROM
    __temp__.ca_disp_events
""",

"ca_FO_dates")
 
    

### Extract case closed events and create flag for earliest/latest event

In [24]:
pydb.create_temp_table(
f"""
Select
    case_number,
    receipt_date,
    entry_date,
    ROW_NUMBER() OVER(PARTITION BY case_number
                       ORDER BY case_number, entry_date ASC) 
      AS min_date,
    ROW_NUMBER() OVER(PARTITION BY case_number
                       ORDER BY case_number, entry_date DESC) 
      AS max_date  
FROM
    {db1}.events
WHERE
  event_model = 'U24'
  AND error = 'N'
  AND mojap_snapshot_date = date '{snapshot_date}'
""",

"ca_case_closed_events")
 

### Extract case reopened events and create flag for earliest/latest event

In [53]:
pydb.create_temp_table(
f"""
Select
    case_number,
    receipt_date,
    entry_date,
    ROW_NUMBER() OVER(PARTITION BY case_number
                       ORDER BY case_number, entry_date ASC) 
      AS min_date,
    ROW_NUMBER() OVER(PARTITION BY case_number
                       ORDER BY case_number, entry_date DESC) 
      AS max_date  
FROM
    {db1}.events
WHERE
  event_model = 'G62'
  AND error = 'N'
  AND mojap_snapshot_date = date '{snapshot_date}'
""",

"ca_case_reopened_events")
 

In [33]:
pydb.read_sql_query ("select * from __temp__.ca_case_reopened_events")

,case_number,receipt_date,entry_date,min_date,max_date
0,MF08P00153,NaT,2009-09-03 13:35:50,4,1
1,MF08P00153,NaT,2009-06-11 09:55:06,3,2
2,MF08P00153,NaT,2009-01-09 11:32:14,2,3
3,MF08P00153,NaT,2008-11-20 12:13:48,1,4


In [72]:
pydb.create_temp_table(
f"""
Select
    t1.case_number,
    (SELECT f.entry_date FROM __temp__.ca_FO_dates f
                         WHERE f.min_date = 1
                         AND f.fo_value = 'Y'
                         AND t1.case_number = f.case_number)
        AS first_final_order,
    (SELECT f.entry_date FROM __temp__.ca_FO_dates f
                         WHERE f.max_date = 1 
                         AND f.fo_value = 'Y'
                         AND t1.case_number = f.case_number)
        AS last_final_order,
    (SELECT f.entry_date FROM __temp__.ca_FO_dates f
                         WHERE f.min_date = 1
                         AND f.fo_value = 'N'
                         AND t1.case_number = f.case_number)
        AS first_no_final_order,
    (SELECT f.entry_date FROM __temp__.ca_FO_dates f
                         WHERE f.max_date = 1 
                         AND f.fo_value = 'N'
                         AND t1.case_number = f.case_number)
        AS last_no_final_order,    
    (SELECT r.entry_date FROM __temp__.ca_case_reopened_events r
                         WHERE r.min_date = 1 
                         AND t1.case_number = r.case_number)
        AS first_reopen,
    (SELECT r.entry_date FROM __temp__.ca_case_reopened_events r
                         WHERE r.max_date = 1 
                         AND t1.case_number = r.case_number)
        AS last_reopen
FROM
  __temp__.ca_disp_events t1
""",
    
"ca_date_test")

In [74]:
pydb.read_sql_query ("select * from __temp__.ca_date_test where case_number = 'ZW15P00150'")

,case_number,first_final_order,last_final_order,first_no_final_order,last_no_final_order,first_reopen,last_reopen
0,ZW15P00150,2015-08-06 15:13:17,2016-10-12 16:55:14,2015-04-24 11:28:12,2016-06-09 16:30:14,2015-09-22 17:00:48,2015-09-22 17:00:48
1,ZW15P00150,2015-08-06 15:13:17,2016-10-12 16:55:14,2015-04-24 11:28:12,2016-06-09 16:30:14,2015-09-22 17:00:48,2015-09-22 17:00:48
2,ZW15P00150,2015-08-06 15:13:17,2016-10-12 16:55:14,2015-04-24 11:28:12,2016-06-09 16:30:14,2015-09-22 17:00:48,2015-09-22 17:00:48
3,ZW15P00150,2015-08-06 15:13:17,2016-10-12 16:55:14,2015-04-24 11:28:12,2016-06-09 16:30:14,2015-09-22 17:00:48,2015-09-22 17:00:48
4,ZW15P00150,2015-08-06 15:13:17,2016-10-12 16:55:14,2015-04-24 11:28:12,2016-06-09 16:30:14,2015-09-22 17:00:48,2015-09-22 17:00:48
5,ZW15P00150,2015-08-06 15:13:17,2016-10-12 16:55:14,2015-04-24 11:28:12,2016-06-09 16:30:14,2015-09-22 17:00:48,2015-09-22 17:00:48
6,ZW15P00150,2015-08-06 15:13:17,2016-10-12 16:55:14,2015-04-24 11:28:12,2016-06-09 16:30:14,2015-09-22 17:00:48,2015-09-22 17:00:48
7,ZW15P00150,2015-08-06 15:13:17,2016-10-12 16:55:14,2015-04-24 11:28:12,2016-06-09 16:30:14,2015-09-22 17:00:48,2015-09-22 17:00:48
8,ZW15P00150,2015-08-06 15:13:17,2016-10-12 16:55:14,2015-04-24 11:28:12,2016-06-09 16:30:14,2015-09-22 17:00:48,2015-09-22 17:00:48
9,ZW15P00150,2015-08-06 15:13:17,2016-10-12 16:55:14,2015-04-24 11:28:12,2016-06-09 16:30:14,2015-09-22 17:00:48,2015-09-22 17:00:48
